In [ ]:
from nbtemplate import display_header, get_path, display_codetoggle
display_header('ImpactL1L2.ipynb')

# Impact of L1 and L2 CAT grating support on effective area and resolving power

In the simplest simulations for Lynx, we did not take into account the L1 and L2 support structures that hold the CAT grating bars in place in detail, we simply reduced the effective area by the geometrical area covered. In the more advanced design work (starting in July 2019) we added a little more realism to the treatment of L1 and L2 support. In particular:

- L1 support bars are not totally opaque. They are thin enough that some hard X-rays pass through, increasing the effective area of the zeroths order compared to the simplest model.
- There is cross dispersion on the L1 bars, which widens the signal in cross-dispersion direction. Those photons can be recovered using a wider extraction region for the astrophysical signal, but at the cost of increased background in the extraction regions. We treat the L1 cross dispersion as if it was independent from the CAT grating. This is an approximation, since the L1 support bars touch the CAT grating and are thus not in the "far field limit".
- The L2 support structures are long enough that the hexagon structures cast shadows for photons that are not propagating exactly parallel to the hexagon sidewalls.
- Diffraction in the L2 hexagons will widen the beam. The structures are so large, that diffraction into higher orders is negligible and we ignore the detailed (hexagon) shape of the L2 support. Instead, we simply widen the beam using the formula for an Airy disk.

In this notebook we compare some simulations with and without the details listed above to study the impact that they have on the Lynx effective area and spectral resolving power. For this comparison, we pick a fiducial Lynx configuration and do not run the comparison over all design options (grating sizes, arrangements, sub-aperturing options).

In [ ]:
import numpy as np
import astropy.units as u
from astropy import table

from marxslynx.simulations import run_monoenergetic_simulation
from marxslynx import lynx
from marxslynx.ralfgrating import facet_table

In [ ]:
from matplotlib import pyplot as plt

%matplotlib inline

In [ ]:
wavegrid = np.arange(0.7, 4., 0.5) * u.nm  # 0.05 steps
energy = wavegrid.to(u.keV, equivalencies=u.spectral())

In [ ]:
instrum = {'2050': lynx.PerfectLynx(conf=lynx.conf)}

In [ ]:
from copy import deepcopy

from marxs.optics import CATGrating
conf_simplecat = deepcopy(lynx.conf)
conf_simplecat['gas_kwargs']['elem_class'] = CATGrating
for k in ['l1_dims', 'l2_dims', 'qualityfactor', 'l1_order_selector']:
    del conf_simplecat['gas_kwargs']['elem_args'][k]

instrum['simplecat'] = lynx.PerfectLynx(conf=conf_simplecat)

In [ ]:
import importlib
import marxslynx.ralfgrating
import marxslynx.lynx
importlib.reload(marxslynx.ralfgrating)
importlib.reload(marxslynx.lynx)

facettab = facet_table(instrum['2050'].elements[2])

In [ ]:
phot_en = []

for i, e in enumerate(energy):
    n = 1e4
    if e.value > 1.25:
        n=5e4
    p = run_monoenergetic_simulation(instrum['2050'], e.value, n)
    p = table.join(p, facettab)
    phot_en.append(p)

In [ ]:
phot_ensimple = []

for i, e in enumerate(energy):
    n = 1e4
    if e.value > 1.25:
        n=5e4
    p = run_monoenergetic_simulation(instrum['simplecat'], e.value, n)
    p = table.join(p, facettab)
    phot_ensimple.append(p)

In [ ]:
from marxs.analysis import resolvingpower_from_photonlist

orders = instrum['2050'].elements[2].elements[0].elements[0].order_selector.orders

def res_power_angle(photons, subaperangle, ang_0=0):
    resolvingpower = np.zeros((len(subaperangle), len(orders)))
    aeff_per_order = np.zeros_like(resolvingpower)
    for i, ang in enumerate(subaperangle):
        ind = np.abs(np.abs(photons['facet_ang']) - ang_0) < ang
        res, width, pos = resolvingpower_from_photonlist(photons[ind], orders, zeropos=0, col='projcirc_y')
        resolvingpower[i, :] = res
        aeff_per_order[i, :] = [photons['probability'][ind & (photons['order'] == o)].sum() for o in orders]
    aeff_per_order = aeff_per_order * instrum['2050'].elements[0].area.to(u.cm**2) / photons.meta['EXPOSURE'][0]
    return resolvingpower, aeff_per_order

In [ ]:
subaperangle = np.linspace(0, np.pi, 7)[1:]
tsubaperangle = np.linspace(0, np.pi/2, 7)[1:]

for p in phot_en:
    ind = p['CCD_ID'] >= 0
    trespow, taeff = res_power_angle(p[ind], tsubaperangle, np.pi/2)
    p.trespow = trespow
    p.taeff = taeff
    respow, aeff = res_power_angle(p[ind], subaperangle)
    p.respow = respow
    p.aeff = aeff

In [ ]:
for p in phot_ensimple:
    ind = p['CCD_ID'] >= 0
    trespow, taeff = res_power_angle(p[ind], tsubaperangle, np.pi/2)
    p.trespow = trespow
    p.taeff = taeff
    respow, aeff = res_power_angle(p[ind], subaperangle)
    p.respow = respow
    p.aeff = aeff

In [ ]:
ang = np.rad2deg(tsubaperangle) * 4

In [ ]:
ind = orders != 0
pres = np.zeros((len(tsubaperangle), len(energy)))
psres = np.zeros_like(pres)

for i in range(len(energy)):
    ps = phot_ensimple[i]
    psres[:, i] = np.ma.average(np.ma.masked_invalid(ps.trespow[:, ind]), 
                          weights=np.ma.masked_invalid(ps.taeff[:, ind]), axis=1)
    ps = phot_en[i]
    pres[:, i] = np.ma.average(np.ma.masked_invalid(ps.trespow[:, ind]), 
                          weights=np.ma.masked_invalid(ps.taeff[:, ind]), axis=1)

## L2 diffraction

In [ ]:
with plt.style.context('fivethirtyeight'):
    fig, ax = plt.subplots(ncols=2, figsize=(10, 6))
    line, = ax[0].plot(ang, psres[:, 3], label='simple')
    line, = ax[0].plot(ang, pres[:, 3], label='L2 diffraction')
    ax[0].legend()
    ax[0].set_ylabel('Resolving power $R$')
    ax[0].set_xlabel('subaperturing angle [deg]')
    # picked slice 0 because that looks almost like it should. Not sure what's going on with the others,
    # but no time to debug now. There are effects that could make it the way it looks, so I don't
    # think it's a bug; it's just that I don't understand what it's telling me.
    ax[1].plot(wavegrid, (pres / psres)[0, :])
    ax[1].set_xlabel('wavelength [nm]')
    ax[1].set_ylabel('$R_{\mathrm{simple}} / R_{\mathrm{incl L2 diffraction}}$')
    fig.suptitle('Diffraction by L2 hexagons', fontsize=40)
    fig.savefig(get_path('figures') + '/L2respos.png', 
            dpi=300, bbox_inches='tight')
    fig.savefig(get_path('figures') + '/L2respow.pdf', bbox_inches='tight')

*Left:* Resolving power for a simple simulation ignoring the L2
diffraction and our approximation for the L2 diffraction. Simulations for
different sub-aperturing angles are shown. Choosing the best sub-aperturing
angle leads to a better $R$. However, for a higher $R$ the L2 diffraction is more
important relative to other effects that broaden the beam. 

*Right:* Ratio
of simulations with and without L2 diffraction for different
wavelengths to verify the theoretically expected wavelength dependence.

In [ ]:
bins = np.arange(634.2, 634.7, 0.02)

fig, ax = plt.subplots(ncols=2, nrows=2)
p = phot_ensimple[3]
ind = (p['order'] == -6) 
scat = ax[0, 0].scatter(p['projcirc_y'][ind], p['projcirc_z'][ind], 
            c=np.rad2deg(p['facet_ang'])[ind])
out = ax[1, 0].hist(p['projcirc_y'][ind], label='simple', bins=bins)
out = ax[1, 1].hist(p['projcirc_y'][ind], weights=p['probability'][ind], label='simple', bins=bins)
#print(np.std(p['projcirc_y'][ind]))
p = phot_en[3]
ind = (p['order'] == -6) 
scat = ax[0, 1].scatter(p['projcirc_y'][ind], p['projcirc_z'][ind], 
            c=np.rad2deg(p['facet_ang'])[ind])

out2 = ax[1, 0].hist(p['projcirc_y'][ind], histtype='step', label='with L1/L2\ndiffraction', bins=bins)
out2 = ax[1, 1].hist(p['projcirc_y'][ind], weights=p['probability'][ind], histtype='step', 
                     label='with L1/L2\ndiffraction', bins=bins)
ax[1,0].legend(loc='upper left')
#print(np.std(p['projcirc_y'][ind]))

In this plot, we look at the distribution of photons for a simple simulation (left) and one that includes L1 and L2 diffraction (right). The top row shows the distribution of photons on the detector (the color denotes the angle of the facet that the photon passed through, so you can see how sub-aperturing would affect the resolving power). 

The bottom row shows histograms of the photons distribution along the dispersion direction for both simulations. Note the very different scales on the y axis for the two plots in the top row. That is due to the L1 diffraction that causes some of the photons to be diffracted perpendicular to the diffraction direction of the main gratings. The histogram is lower in the simulation that includes L1 and L2 effects, because both L1 and L2 structures absorb some light. Although the peak is lower, the histogram has the almost the same width, in other words, it has a larger FWHM. This is the effect of the L2 diffraction.

In [ ]:
photons = phot_en[3][phot_en[3]['probability'] > 0]
plt.scatter(photons['projcirc_y'], photons['projcirc_z'], c=photons['order'])
plt.colorbar(label='Diffraction order')
plt.xlabel('Dispersion direction in focal plane [mm]')
plt.ylabel('Cross-dispersion direction [mm]')

In [ ]:
photonss = phot_ensimple[3][phot_ensimple[3]['probability'] > 0]
plt.scatter(photonss['projcirc_y'], photonss['projcirc_z'], c=photonss['order'])
plt.colorbar(label='Diffraction order')
plt.xlabel('Dispersion direction in focal plane [mm]')
plt.ylabel('Cross-dispersion direction [mm]')

The two plots above show the effect of the L1 diffraction on the pattern globally. The "fish" shaped lower plot is fro ma simulation without L1 diffraction. The Rowland configuration optimizes the width of the spot in dispersion direction, at the cost of the width in cross-dispersion direction. Only in two spots (where the torus intersect itself) is the width in cross-dispersion direction also small. The first of those is at the focal point, where the zero order photons are seen, the second one around 600 mm. Since this is a monoenergetic simulation, we see discrete orders and not a band of signal.

The upper plot included L1 diffraction. Note that the range on the y-axis of the plot is much wider. The fish-shaped structure in the middle can still be made out, but now every spot is repeated above and below as a results of L1 dispersion.

In [ ]:
from astropy.visualization import (MinMaxInterval, LogStretch,
                                   ImageNormalize)

bins = [np.linspace(500, 700, 200), np.linspace(-20, 20, 100)]

photonss = phot_ensimple[3][phot_ensimple[3]['probability'] > 0]
Hs, xe, ye = np.histogram2d(photonss['projcirc_y'], photonss['projcirc_z'], weights=photonss['probability'],
               bins=bins)

photons = phot_en[3][phot_en[3]['probability'] > 0]
H, xe, ye = np.histogram2d(photons['projcirc_y'], photons['projcirc_z'], weights=photons['probability'],
               bins=bins)
# Create an ImageNormalize object
norm = ImageNormalize(H, interval=MinMaxInterval(), stretch=LogStretch())

with plt.style.context('fivethirtyeight'):
    fig, ax = plt.subplots(nrows=2, figsize=(10, 6))
    im1 = ax[0].imshow(Hs.T, norm=norm, origin='lower', extent=(xe[0], xe[-1], ye[0], ye[-1]),
                       aspect='auto', cmap=plt.get_cmap('magma'))
    im2 = ax[1].imshow(H.T, norm=norm, origin='lower', extent=(xe[0], xe[-1], ye[0], ye[-1]),
                       aspect='auto', cmap=plt.get_cmap('magma'))
    cbar1=plt.colorbar(im1, ax=ax[0], ticks=[0, 10, 30, 100, 300])
    cbar2=plt.colorbar(im2, ax=ax[1], ticks=[0, 10, 30, 100, 300])
    ax[0].grid(False)
    ax[1].grid(False)
    ax[0].set_title('simple CAT grating')
    ax[1].set_title('including L1 diffraction')
    
#plt.colorbar(label='Diffraction order')
plt.xlabel('Dispersion direction in focal plane (mm)')
plt.text(480, 65, 'Cross-dispersion direction (mm)', rotation=90, fontsize=20)

fig.subplots_adjust(hspace=.3)
fig.savefig(get_path('figures') + '/L1img.png', 
            dpi=300, bbox_inches='tight')
fig.savefig(get_path('figures') + '/L1img.pdf', bbox_inches='tight')
print(wavegrid[3])

This plot focuses on two orders that are seen on the detector. It shows the same effect as above, but instead of plotting every ray as a point, this figures bins it up on the detector making use of the probability of each ray.

In [ ]:
display_codetoggle()